# Two-points calibration

Notebook which computes and saves results of the two-points calibration

In [ ]:
import numpy as np
import importlib
import matplotlib.pyplot as plt
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_plot_function as theodo_p
import scripts.theodolite_function as theodo_f
import scripts.resection_functions as theodo_r

In [ ]:
# This cell gathers result for the two-points resection

theodo_r = importlib.reload(theodo_r)
theodo_u = importlib.reload(theodo_u)
theodo_f = importlib.reload(theodo_f)

pilier_A = np.array([0, 0, 0, 1]).T
pilier_B = np.array([0, np.sqrt(181.016**2 - 0.323**2), 0.323, 1])
pilier_C = np.array([0, np.sqrt(363.492**2 - 1.038**2), 1.038, 1])
pilier_D = np.array([0, np.sqrt(548.578**2 - 2.374**2), 2.374, 1])
pilier_ref = [pilier_A, pilier_B, pilier_C, pilier_D]

path_list = [
    "../data/20220711-1/",
    "../data/20220715-1/"
]

path_list_inter = [
    ["../data/20220711-2/"],
    [""]
]

save_results = False

error_distance_resection = []
error_distance_resection_exp = []
error_distance_resection_cp = []
TF_list_distance_resection = []
for i,j in zip(path_list,path_list_inter):
    # Generate file path
    file_geomatic = i + 'total_stations/Two_points.txt'
    file_GCP = theodo_u.if_file_exist(i + 'total_stations/GCP.txt','')
    error_distance_loop = []
    error_distance_exp_loop = []
    error_distance_cp_loop = []
    Tf_distance_loop = []
    for m in j:
        file_prediction = theodo_u.if_file_exist(m + 'filtered_prediction/f-2-1-1-1-6-0-L_','1.csv')
        file_sensors = theodo_u.if_file_exist(m+"sensors_extrinsic_calibration/calibration_results.csv",'')
        extrinsic_calibration_results = theodo_u.read_extrinsic_calibration_results_file(file_sensors)
        # Compute error of two-points resection
        TF1,TF2,TF3,error_all_distance_resection,error_distance_cp,error_distance_resection_experiment = theodo_r.geomatic_resection_errors_angle_based(file_geomatic, pilier_ref, file_prediction, extrinsic_calibration_results, file_GCP)
        error_distance_loop += error_all_distance_resection
        error_distance_exp_loop += error_distance_resection_experiment
        error_distance_cp_loop += error_distance_cp
        Tf_distance_loop += [TF1, TF2, TF3]
    error_distance_resection += error_distance_loop
    error_distance_resection_exp += error_distance_exp_loop
    error_distance_resection_cp += error_distance_cp_loop
    TF_list_distance_resection += Tf_distance_loop
    print("Cp error: ",np.median(error_distance_cp_loop),np.std(error_distance_cp_loop))
    print("Inter-prism: ", np.median(error_distance_exp_loop),np.std(error_distance_exp_loop))
    # Save results if wanted
    if(save_results):
        theodo_u.save_error_list_to_file(error_distance_cp_loop, i+"list_error/error_distance_resection_cp.csv")
        theodo_u.save_error_list_to_file(error_distance_exp_loop, i+"list_error/error_distance_resection_experiment.csv")
        theodo_u.save_tf_list_to_file_multi(Tf_distance_loop, i+"list_tf/TF_list_distance_resection.csv")


In [ ]:
# Display results according to different metrics
%matplotlib notebook

fig = plt.figure(figsize =(10, 5))
ax = fig.add_subplot(111)
lis = [error_distance_resection,error_distance_resection_cp,error_distance_resection_exp]

box = ax.boxplot(lis, notch=True, patch_artist=True, vert = 1, showfliers=False, showmeans=False)
plt.xticks([1,2,3],
               ["Two-points \n\n\n M: "+str(round(np.median(lis[0]),2))+"mm \n Std: "+str(round(np.std(lis[0]),2))+"mm\n"+str(round(len(lis[0])/3))+" pts",
            "GCP\n\n\n M: "+str(round(np.median(lis[1]),2))+"mm \n Std: "+str(round(np.std(lis[1]),2))+"mm\n"+str(round(len(lis[1])/3))+" pts",
            "Inter-prism\n\n\n M: "+str(round(np.median(lis[2]),2))+"mm \n Std: "+str(round(np.std(lis[2]),2))+"mm\n"+str(round(len(lis[2])/3))+" pts",
           ])
colors_box = ['#069AF3','#069AF3','#069AF3']

for patch, color in zip(box['boxes'], colors_box):
    patch.set_facecolor(color)

ax.set_ylabel("Error [mm]")
fig.tight_layout()
plt.show()